In [ ]:
import pandas as pd 
import numpy as np 
from keras.models import Sequential 
from keras.layers.core import Flatten, Dense, Dropout
from tensorflow.python.keras.layers import Input, LSTM, Bidirectional, Dense, Embedding
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
import cv2
import tensorflow as tf
import os
import tensorflow_datasets as tfds
from tensorflow.data.experimental import make_csv_dataset
from keras.utils.np_utils import to_categorical
from tensorflow import io
from tensorflow.train import BytesList, FloatList, Int64List
from tensorflow.train import Example, Features, Feature

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')  
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

In [ ]:
num_features = 64
num_labels = 7
batch_size = 64
epochs = 50
width, height = 48, 48

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!curl https://sdk.cloud.google.com | bash

In [ ]:
!gcloud init --skip-diagnostics

In [ ]:
def make_example(image, label):

  image = np.array(image.split(" "), 'float32')

  image_feature = Feature(
        bytes_list=BytesList(value=[
            tf.io.serialize_tensor(image).numpy(),
        ])
    )
  label_feature = Feature(
        int64_list=Int64List(value=[
            label,
        ])
    )

  features = Features(feature={
        'image': image_feature,
        'label': label_feature,
    })
    
  example = Example(features=features)
    
  return example.SerializeToString()

In [ ]:
csv = pd.read_csv("gs://assignment2-eb9e3.appspot.com/fer2013.csv").values
out = "gs://assignment2-eb9e3.appspot.com/csv.tfrecords"
with tf.io.TFRecordWriter(out) as writer:
    for row in csv:
        writer.write(make_example(row[1], row[0]))
writer.close()

In [ ]:
GCS_PATH = "gs://assignment2-eb9e3.appspot.com/csv.tfrecords"

In [ ]:
def decode_image(image_data):
    image = tf.io.decode_raw(image_data, tf.uint8)
    #image = tf.strings.split(image, sep=' ')
    
    image = tf.cast(image, tf.float32) / 255.0  # convert image to floats in [0, 1] range

    image = tf.reshape(image, [48, 48, 3]) # explicit size needed for TPU
    return image

def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "class": tf.io.FixedLenFeature([], tf.int64, default_value=0),  # shape [] means single element
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tf.cast(example['class'], tf.int64)
    return image, label # returns a dataset of (image, label) pairs

def load_dataset(filenames, labeled=True, ordered=False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # disregarding data order. Order does not matter since we will be shuffling the data anyway.

    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed

    dataset = tf.data.TFRecordDataset(filenames) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord)
    # returns a dataset of (image, label) pairs if labeled=True or (image, id) pairs if labeled=False
    return dataset

In [ ]:
def data_augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.rot90(image, k=1) # rotate 90º
    image = tf.image.random_brightness(image, max_delta=.2)
    return image, label

In [ ]:
ds = load_dataset(GCS_PATH)

In [ ]:
ds = ds.map(data_augment)
ds = ds.repeat()
ds = ds.shuffle(2048)
ds = ds.batch(64)
ds = ds.cache()
ds = ds.prefetch(64)

In [ ]:
ds

<PrefetchDataset shapes: ((None, 48, 48, 3), (None,)), types: (tf.float32, tf.int64)>

In [ ]:
def create_model():
  return tf.keras.Sequential(
      [
       tf.keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=( ,48, 48, 1)),
       tf.keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu'),
       tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)),
       tf.keras.layers.Dropout(0.5),

       tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'),
       tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'),
       tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)),
       tf.keras.layers.Dropout(0.5),

       tf.keras.layers.Conv2D(128, kernel_size=(3,3), activation='relu'),
       tf.keras.layers.Conv2D(128, kernel_size=(3,3), activation='relu'),
       tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)),

       tf.keras.layers.Flatten(),

       tf.keras.layers.Dense(1024, activation='relu'),
       tf.keras.layers.Dropout(0.2),
       tf.keras.layers.Dense(1024, activation='relu'),
       tf.keras.layers.Dropout(0.2),
       tf.keras.layers.Dense(num_labels, activation='softmax')])

In [ ]:
strategy = tf.distribute.TPUStrategy(resolver)
with strategy.scope():
  model = create_model()
  model.compile(optimizer='sgd',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                metrics=['accuracy'])

In [ ]:
model.fit(ds, use_multiprocessing= True, steps_per_epoch = 50)

InvalidArgumentError: ignored

In [ ]:
with strategy.scope():
  model = create_model()
  model.compile(optimizer='sgd',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

batch_size = 64
steps_per_epoch = 50 // batch_size
validation_steps = 50 // batch_size

model.fit(x_train, train_y,
          epochs=5,
          steps_per_epoch=steps_per_epoch,
          verbose = 1,
          shuffle = True,
          validation_data=(x_test, test_y))

In [ ]:
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, Activation
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from tensorflow.keras.losses import categorical_crossentropy 
from tensorflow.keras import layers
from tensorflow import keras    #aşağıya inme

In [ ]:
model = Sequential()

model.add(Conv2D(64, kernel_size=(3,3), activation='relu', input_shape=(x_train.shape[1:])))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
#model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.5))

#2nd Convolutional Layer
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(Conv2D(64, (3,3), activation='relu'))
#model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.5))

#3rd Convolutional Layer
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(Conv2D(128, (3,3), activation='relu'))
#model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Flatten())

#fully connected neural networks
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(num_labels, activation='softmax'))

#model.summary()



In [ ]:
model.compile(loss=categorical_crossentropy, optimizer= 'sgd', metrics= ['accuracy'])

In [ ]:
#Training the model
model.fit(x_train, train_y, batch_size=batch_size, epochs=epochs, verbose=1, validation_data= (x_test, test_y), shuffle= True)

Epoch 1/50
449/449 [==============================] - 505s 1s/step - loss: 1.8542 - accuracy: 0.2365 - val_loss: 1.8093 - val_accuracy: 0.2519
Epoch 2/50
449/449 [==============================] - 481s 1s/step - loss: 1.7927 - accuracy: 0.2532 - val_loss: 1.7977 - val_accuracy: 0.2497
Epoch 3/50
364/449 [=======================>......] - ETA: 1:28 - loss: 1.7786 - accuracy: 0.2587

In [ ]:
#Saving the model to use it later on
fer_json = model.to_json()  
with open("fer.json", "w") as json_file:  
    json_file.write(fer_json)  
model.save_weights("fer.h5")  

NameError: ignored

In [ ]:
import os 
import cv2
import numpy as np
from keras.models import model_from_json
from keras.preprocessing import image 

In [ ]:
scores = model.evaluate(x_test, test_y, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 55.34%


In [ ]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
module_wrapper_17 (ModuleWra (None, 46, 46, 64)        640       
_________________________________________________________________
module_wrapper_18 (ModuleWra (None, 44, 44, 64)        36928     
_________________________________________________________________
module_wrapper_19 (ModuleWra (None, 22, 22, 64)        0         
_________________________________________________________________
module_wrapper_20 (ModuleWra (None, 22, 22, 64)        0         
_________________________________________________________________
module_wrapper_21 (ModuleWra (None, 20, 20, 64)        36928     
_________________________________________________________________
module_wrapper_22 (ModuleWra (None, 18, 18, 64)        36928     
_________________________________________________________________
module_wrapper_23 (ModuleWra (None, 9, 9, 64)         

In [ ]:
def get_metrics(predictions,targets):
    # Calculate metrics
    # Accuarcy
    acc = np.mean(np.equal(np.argmax(predictions,1),np.argmax(targets,1)))
    # Confusion matrix
    conf = confusion_matrix(np.argmax(targets,1),np.argmax(predictions,1))     
    # Class weighted accuracy
    wacc = conf.diagonal()/conf.sum(axis=1)  
    # Auc
    fpr = {}
    tpr = {}
    roc_auc = np.zeros([numClasses])
    for i in range(numClasses):
        fpr[i], tpr[i], _ = roc_curve(targets[:, i], predictions[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])       
    # F1 Score
    f1 = f1_score(np.argmax(predictions,1),np.argmax(targets,1),average='weighted')        
    # Print
    print("Accuracy:",acc)
    print("F1-Score:",f1)
    print("WACC:",wacc)
    print("Mean WACC:",np.mean(wacc))
    print("AUC:",roc_auc)
    print("Mean Auc:",np.mean(roc_auc))        
    return acc, f1, wacc, roc_auc

In [ ]:
model = model_from_json(open("fer.json", "r",), read())
model.load_weights('fer.h5')
face_hear_cascade = cv2.CascadeClassifier

NameError: ignored